# Gangavstand fra sentrumssone

Notebook som definerer *gangavstandbuffrede* sentrumssoner for områder med byvekstavtale eller belønningsordning. 

Hver kommune har ett sentrumsområde, det største.

In [1]:
import geopandas as gpd
import pandas as pd
import polars as pl
import pydeck as pdk
import osmnx as ox
import networkx as nx 
from lonboard import viz
from os import walk
from fiona import listlayers
from pathlib import Path
import re

In [3]:
crs_norge = "EPSG:25833"
crs_plot = "EPSG:4326"

## Definere områdene

In [ ]:
områder_path = Path("geografiske_områder")
område_gdf_list = []

for root, dirs, files in walk(områder_path):
    for f in files:
        if f.endswith("geojson"):
            l = listlayers(f"{root}/{f}")
            ll = [f for f in l if f.lower().endswith("kommune")]
            if len(ll)==1:
                ll = ll[0]
                område_gdf_list.append(
                    gpd.read_file(f"{root}/{f}", layer=ll).assign(
                        område = root.removeprefix("C:\\Users\\josche\\Documents\\python\\projects\\arealgruppe\\geografiske_områder\\"), 
                        kommune = f
                        )
                    )
            else:
                print(f"problem med {root}/{f} og layers {l}. finn på noe smartere!")

In [5]:
områder_gdf = pd.concat([gdf.to_crs(crs_norge) for gdf in område_gdf_list], ignore_index=True)
områder_gdf = områder_gdf[["område", "geometry", "kommune"]]
områder_gdf["kommune"] = [re.search(r'(?<=\d_).+?(?=_\d)', x)[0] for x in områder_gdf["kommune"]]

In [6]:
områder_gdf.head(2)

,område,geometry,kommune
0,Bergensområdet,"POLYGON ((-40889.251 6732715.54, -41185.821 67...",Bergen
1,Bergensområdet,"POLYGON ((-39434.611 6710972.25, -36231.251 67...",Os


## Sentrumssoner

### Oslo

In [7]:
indre_by_øst = ["Gamle Oslo", "Grünerløkka", "Sagene"]
indre_by_vest = ["St. Hanshaugen", "Frogner", "Sentrum"]
oslo_indre_by_bydeler = indre_by_øst + indre_by_vest

In [ ]:
# fil fra https://experience.arcgis.com/experience/b5942497b01d4c6496ed440c3af0a397
oslo_bydeler = gpd.read_file("Bydeler_4465115856483287627.geojson")
oslo_indre_by = oslo_bydeler[oslo_bydeler["BYDELSNAVN"].str.contains("|^".join(oslo_indre_by_bydeler))]
oslo_indre_by

,FID,kommunenum,BYDEL,BYDELSNAVN,geometry
2,3,0301,03,Sagene,"POLYGON ((1199084.696 8384380.16, 1199061.532 ..."
9,10,0301,05,Frogner,"POLYGON ((1193352.519 8384856.068, 1193416.92 ..."
10,11,0301,01,Gamle Oslo,"POLYGON ((1203243.062 8381163.223, 1203277.835..."
11,12,0301,02,Grünerløkka,"POLYGON ((1199869.689 8384871.058, 1199878.828..."
13,14,0301,16,Sentrum,"POLYGON ((1194258.77 8378193.217, 1194428.592 ..."
14,15,0301,04,St. Hanshaugen,"POLYGON ((1195366.275 8381485.949, 1195331.901..."


### Ikke-Oslo 

In [ ]:
sentrumssoner = gpd.read_parquet("sentrumssoner2025.parquet")
sentrumssoner = sentrumssoner.sjoin(områder_gdf.to_crs(sentrumssoner.crs), how="inner", predicate="within").reset_index(drop=True)
sentrumssoner.head(2)

,ogc_fid,komm_nr,tett_nr,hovedsentrum,areal,stataar,opphav,objtype,dato,tettstedsnavn,kommunenavn,__index_level_0__,geom,index_right,område,kommune
0,0,0301,0801,A,110235.967991,2025,SSB,Sentrumssone,20250101,Oslo,Oslo,0,"POLYGON ((255964.557 6650050.437, 255964.325 6...",46,Osloområdet,Oslo
1,1,0301,0801,A,347054.678353,2025,SSB,Sentrumssone,20250101,Oslo,Oslo,1,"POLYGON ((258095.211 6650498.92, 258095.204 66...",46,Osloområdet,Oslo


In [10]:
kommunesentrum = (
    sentrumssoner
    .query("kommunenavn!='Oslo'") # Egen definisjon
    .query("not (kommunenavn=='Sola' & tettstedsnavn=='Kolnes')") # Solas største tettsted er flyplassen
    .sort_values("areal", ascending=False)
    .drop_duplicates(subset="kommunenavn")
)

In [12]:
kommunesentrum = kommunesentrum[["område", "tettstedsnavn", "kommunenavn","areal","geom"]]
kommunesentrum.head(5)

,område,tettstedsnavn,kommunenavn,areal,geom
253,Bergensområdet,Bergen,Bergen,1.639143e+06,"POLYGON ((-32957.864 6735346.367, -32957.876 6..."
298,Trondheimsområdet,Trondheim,Trondheim - Tråante,1.115273e+06,"POLYGON ((269532.874 7041840.334, 269532.859 7..."
84,Nord-Jæren,Stavanger/Sandnes,Stavanger,1.006543e+06,"POLYGON ((-32689.341 6574333.226, -32689.345 6..."
219,Kristiansandsområdet,Kristiansand,Kristiansand,6.848087e+05,"POLYGON ((87417.663 6466321.828, 87417.632 646..."
143,Osloområdet,Oslo,Lillestrøm,5.094358e+05,"POLYGON ((278881.112 6652845.59, 278881.082 66..."


In [20]:
sentrumssoner_område = pd.concat(
    [
        kommunesentrum.rename(columns={"geom":"geometry"}),
        gpd.GeoDataFrame({"område":"Osloområdet", "tettstedsnavn":"Oslo", "kommunenavn":"Oslo", "areal":oslo_indre_by.dissolve()["geometry"].area}, geometry=oslo_indre_by.dissolve()["geometry"]).to_crs(kommunesentrum.crs)
    ]
    ).sort_values(["område", "areal"]).reset_index(drop=True)

sentrumssoner_område = sentrumssoner_område.set_geometry("geometry")

## Gangavstand fra sentrumssoner

Bruke 1000 m


Endre på Sola

In [21]:
sentrumssoner_område.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°E and 18°E: Austria; Denmark - offshore and offshore; Germany - onshore and offshore; Norway including Svalbard - onshore and offshore.
- bounds: (12.0, 46.4, 18.01, 84.42)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
gangvei_fra_sentrum_list = []

# Define buffer distances once
BUFFER_START_NODES = 10  # meters
TRIP_DISTANCE = 1000     # meters
DOWNLOAD_BUFFER = 2500  # meters

for _, row in sentrumssoner_område.iterrows():       
    print(f"Processing: {row['kommunenavn']}")
    
    # 1. Geometry Setup
    # Ensure we use the geometry directly. .item() converts to a shapely object.
    center_geom = row["geometry"]
    row_polygon_series = gpd.GeoSeries([center_geom], crs=crs_norge)
    
    # Create the download area in WGS84 for OSMnx
    download_poly = row_polygon_series.buffer(DOWNLOAD_BUFFER).to_crs(crs_plot).iloc[0]

    try:
        # 2. Optimized Network Loading
        # Combine types in one call if possible, or use 'all' and filter
        # Here we use 'all_private' or custom filter to get both walk and bike in one go
        cf = '["highway"~"cycleway|footway|path|pedestrian|living_street|residential|service"]'
        G = ox.graph_from_polygon(download_poly, network_type='all', custom_filter=cf, simplify=True)
        
        # Project once
        G_proj = ox.project_graph(G, to_crs=crs_norge)
        nodes_gdf = ox.graph_to_gdfs(G_proj, edges=False)

        # 3. Spatial Query (Using spatial index for speed)
        # We find nodes within 10m of the center polygon
        search_area = center_geom.buffer(BUFFER_START_NODES)
        possible_indexes = nodes_gdf.sindex.query(search_area, predicate="intersects")
        start_nodes = nodes_gdf.iloc[possible_indexes].index.tolist()

        if not start_nodes:
            print(f"No start nodes found for {row['kommunenavn']}, skipping...")
            continue

        # 4. Isochrone Analysis (Multi-source Dijkstra)
        node_distances = nx.multi_source_dijkstra_path_length(
            G_proj, 
            sources=start_nodes, 
            cutoff=TRIP_DISTANCE, 
            weight='length'
        )

        # 5. Result Extraction
        accessible_nodes = list(node_distances.keys())
        # Filter edges where both u and v are in the accessible set
        sub_G = G_proj.subgraph(accessible_nodes)
        
        if len(sub_G.edges) > 0:
            edges_gdf = ox.graph_to_gdfs(sub_G, nodes=False).copy()
            
            edges_gdf = edges_gdf.assign(
                kommunenavn = row["kommunenavn"],
                tettstedsnavn = row["tettstedsnavn"]
            )
            
            gangvei_fra_sentrum_list.append(edges_gdf)
            
    except Exception as e:
        print(f"Error processing {row['kommunenavn']}: {e}")

# Combine all results into one GeoDataFrame
if gangvei_fra_sentrum_list:
    final_gdf = pd.concat(gangvei_fra_sentrum_list, ignore_index=True)

Processing: Askøy
Processing: Bjørnafjorden
Processing: Øygarden
Processing: Alver
Processing: Bergen
Processing: Lier
Processing: Øvre Eiker
Processing: Kongsberg
Processing: Drammen
Processing: Porsgrunn
Processing: Skien
Processing: Birkenes
Processing: Lillesand
Processing: Vennesla
Processing: Kristiansand
Processing: Sarpsborg
Processing: Fredrikstad
Processing: Randaberg
Processing: Sola
Processing: Sandnes
Processing: Stavanger
Processing: Enebakk
Processing: Nittedal
Processing: Gjerdrum
Processing: Nesodden
Processing: Eidsvoll
Processing: Vestby
Processing: Aurskog-Høland
Processing: Frogn
Processing: Nes
Processing: Ås
Processing: Nordre Follo
Processing: Lørenskog
Processing: Asker
Processing: Ullensaker
Processing: Bærum
Processing: Lillestrøm
Processing: Oslo
Processing: Tromsø
Processing: Malvik
Processing: Skaun
Processing: Orkland
Processing: Melhus
Processing: Stjørdal
Processing: Trondheim - Tråante


In [ ]:
final_gdf['highway'] = final_gdf['highway'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
cols_to_save = ["highway", "length", "geometry", "kommunenavn", "tettstedsnavn"]
final_gdf[cols_to_save].to_parquet("gangvei_fra_sentrumssoner.parquet")

In [33]:
df_buf = final_gdf.copy()
gdf_buf = df_buf.set_geometry(final_gdf.buffer(50))  
corridor_per_kommune = gdf_buf.dissolve(by="kommunenavn", as_index=False)

In [66]:
centers = sentrumssoner_område.to_crs(corridor_per_kommune.crs).rename(columns={"geom":"geometry"}).copy()
centers = centers[["kommunenavn", "geometry"]]
centers = centers.set_geometry("geometry")

stacked = gpd.GeoDataFrame(
    pd.concat([corridor_per_kommune[["kommunenavn", "geometry"]], centers], ignore_index=True),
    geometry="geometry",
    crs=corridor_per_kommune.crs
)
result = stacked.dissolve(by="kommunenavn", as_index=False)

In [71]:
result = result.merge(sentrumssoner_område[["område", "kommunenavn"]].drop_duplicates(), on="kommunenavn", how="left")

In [72]:
result.to_file("buffrede_sentrumssoner.gpkg", driver="GPKG")

In [73]:
result.head(2)

,kommunenavn,geometry,område
0,Alver,"POLYGON ((-31824.615 6751283.654, -31818.243 6...",Bergensområdet
1,Asker,"POLYGON ((243020.69 6641132.241, 243016.412 66...",Osloområdet
